In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import matchzoo as mz

Using TensorFlow backend.


In [2]:
tf.__version__

'2.2.0'

In [44]:
test_articles = pd.read_csv('../dataset/data_articles_test.csv', delimiter=',')

In [4]:
indexing_distinct_tables.indices.close(index='distinct_tables')
indexing_distinct_tables.indices.put_settings(index='distinct_tables', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexing_distinct_tables.indices.open(index='distinct_tables')

{'acknowledged': True, 'shards_acknowledged': True}

In [ ]:
# "tablePgID"
# "tablePgTitle"
# "tablePgFullText"
# "tablePgMetaDescription"
# "tablePgSummary"
# "tablePgKeywords"
# "tableSectionTitle"
# "tableCaption"
# "tableHeader"
# "tableBody"

In [5]:
def search_indexing(query):
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["tablePgFullText"] 
            }
        }
    })
    
    return result

In [6]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [7]:
def search_index(query):
    
    tables_index = []

    result_index = search_indexing(query)
        
    for hit in result_index['hits']['hits']:
    
        table_ID = hit['_source']['tablePgID']
        
        table_page_title = hit['_source']['tablePgTitle']
    
        tables_index.append([table_ID,table_page_title])
    
    return tables_index

In [9]:
ranking_model = mz.load_model('DSSM68')

In [10]:
train_dataset = pd.read_csv('../train_data/train_data_1_1', delimiter=',')
train_dataset = train_dataset.replace(np.nan, ' ', regex=True)

In [11]:
list_data1 = []
for i, row in train_dataset.iterrows():
    
    line1 = {'id_left': str(row['article_id']),
            'text_left':str(row['article_page_title']),
            'id_right':str(row['table_id']),
            'text_right':str(row['table_page_title']),
            'label':row['label']
           }
    
    list_data1.append(line1)

df1 = pd.DataFrame(list_data1)
train_pack = mz.pack(df1)

In [12]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
train_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 44853/44853 [00:04<00:00, 9131.89it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 68727/68727 [00:05<00:00, 12444.13it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2885474/2885474 [00:00<00:00, 5102126.44it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 44853/44853 [00:16<00:00, 2650.59it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 68727/68727 [00:27<00:00, 2488.82it/s]


In [46]:
result = []
    
def run_search(k):
    
    TOP_K = k
    accuracy = []

    for i, row in tqdm(test_articles.iterrows()):

        article_ID = row['page_id']
        article_title_text = row['page_title']
        query = row['page_title']+" "+row['meta_description']+" "+row['keywords']

        #searching over index
        ranked_tables_index = search_index(query)

        ranked_tables_model = []        
        class_list = []

        if len(ranked_tables_index) > 0:

            for table_ID, table_title in (ranked_tables_index):

                line = {'id_left': article_ID,
                         'text_left':article_title_text,
                         'id_right':table_ID,
                         'text_right':table_title
                        }

                class_list.append(line)

            df2 = pd.DataFrame(class_list)
            test_pack = mz.pack(df2)
            valid_processed = preprocessor.transform(test_pack)
            test_x, test_y = valid_processed.unpack()


            table_ranking_model = ranking_model.predict(test_x)

            for i in range(0,len(table_ranking_model)):

                ranked_tables_model.append([ranked_tables_index[i][0],ranked_tables_index[i][1],table_ranking_model[i][0]]) 

            data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
            data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)   
            final_ranked_tables = data_frame_sorting.iloc[0:TOP_K,0:1].values

            selected_top = data_frame_sorting.head(TOP_K)
            min_score = selected_top['table_ranking'].min()
            draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
            final_ranked_tables = draw_tables_socres.iloc[:,0:1].values

            accuracy.append(get_accuracy(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(k),str(round(np.mean(accuracy),4))])

In [47]:
accuracy_K = [1,5,10,20]

for k in accuracy_K:

    run_search(k)

0it [00:00, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 52.93it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
1it [00:00,  2.50it/s]with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 3159.88it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashin

Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
12it [00:11,  1.21it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 3300.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 829.41it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
13it [00:12,  1.14it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 355.90it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
24it [00:18,  1.97it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 2328.59it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████

Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
35it [00:41,  1.35it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 3942.05it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 363.33it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
36it [00:42,  1.57it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 399.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
47it [00:50,  2.17it/s]ith chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 2235.40it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████

...

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 1233.98it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
971it [01:44,  9.24it/s]th chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 4627.94it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|█████████

Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
982it [01:46,  9.44it/s]th chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 4761.98it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 792.42it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
983it [01:46,  9.27it/s]th chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 1155.14it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
994it [01:47,  9.24it/s]th chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 4714.66it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|█████████

Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
1005it [01:48,  9.42it/s]h chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 4819.27it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 1222.12it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
1006it [01:48,  9.42it/s]h chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|█████

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 1/1 [00:00<00:00, 1174.55it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/100 [00:00<?, ?it/s]
1017it [01:49,  9.45it/s]h chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|██████████| 100/100 [00:00<00:00, 4626.82it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing:   0%|          | 0/1 [00:00<?, ?it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter => WordHashing: 100%|█████████

In [48]:
result

[['Acc@1', '0.0097'],
 ['Acc@5', '0.0507'],
 ['Acc@10', '0.0975'],
 ['Acc@20', '0.2027']]